In [ ]:
# Install dependencies
!pip install -q --upgrade diffusers transformers scipy ftfy huggingface_hub
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt
%cd /content/yolov7
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

In [2]:
# [2] Import modules

import torch
import os
import glob
from diffusers import StableDiffusionPipeline
from huggingface_hub import notebook_login
from IPython.display import Image, display

In [3]:
# [3] Create a folder for storing test dataset

testsets = 'testsets'
os.mkdir(testsets)

In [ ]:
# [1] Log-in to Hugging Face

notebook_login()

In [ ]:
# [2] Prepare Stable Diffusion model

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16")
pipe = pipe.to("cuda")

In [ ]:
# [3] Generate test image

prompt = "mining equipment under the sea" # <--- your prompt
image = pipe(prompt).images[0]

image_name = f"{testsets}/test.png"
image.save(image_name)

display(Image(filename=image_name, width=158, height=158))

In [ ]:
# [4] Prepare function to generate images en-masse and store them in a folder

def generate_images(
    prompt, 
    num_images_to_generate, 
    num_images_per_prompt,
    guidance_scale=4,
    output_dir="billboards",
    display_images=False,
):

    num_iterations = num_images_to_generate
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_iterations):
        images = pipe(
            prompt, num_images_to_generate=num_images_to_generate, guidance_scale=guidance_scale
        )
        for idx, image in enumerate(images.images):
            image_name = f"{output_dir}/billboard-{(i*num_images_per_prompt)+idx}.png"
            image.save(image_name)
            if display_images:
                display(Image(filename=image_name, width=128, height=128))

In [ ]:
# [5] Generate and save images

generate_images("billboard on a busy street in Indonesia", 3, 1, guidance_scale=4, display_images=True)